In [143]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('MyApp').getOrCreate()

In [144]:
"""
Loading the training csv.
"""

training = spark.read.option("escape", "\"").csv('../datasets/imdb/train_movies_extra_data_new.csv', header=True, inferSchema=True)

In [145]:
"""
Loading the validation csv.
"""

validation = spark.read.option("escape", "\"").csv('../datasets/imdb/validation_extra_data.csv', header=True, inferSchema=True)

In [146]:
"""
Loading the test csv.
"""

test = spark.read.option("escape", "\"").csv('../datasets/imdb/test_extra_data.csv', header=True,
    inferSchema=True)

In [147]:
from pyspark.sql.functions import col, udf, when, split, regexp_replace, min, explode, lower, array_distinct, array_contains
from pyspark.sql.types import BooleanType, IntegerType
from pyspark.ml.feature import OneHotEncoder, StringIndexer

import re

def manage_awards(award):

    if award is not None:
        if re.search('win', award) or re.search('wins', award) or re.search('won', award):
            return 1
        else:
            return 0
    else:
        return False

my_udf = udf(manage_awards, IntegerType())

def preprocessing_method(df):

    """
    This method will convert to integer all the years and will put to categorical weather
    a movie won any awards or not.
    :param df: dataframe
    :return: return a preprocessed dataframe
    """

    """
    Here casting to integer all the numerical values
    """
    df = df.withColumn('startYear', col('startYear').cast('int'))
    df = df.withColumn('endYear', col('endYear').cast('int'))
    df = df.withColumn('numVotes', col('numVotes').cast('int'))
    df = df.withColumn('runtimeMinutes', col('runtimeMinutes').cast('int'))

    """
    Preprocess boxoffice, and put the lowest value if it is null.
    """

    df = df.withColumn("boxoffice", regexp_replace(col("boxoffice"), "[$,]", "").cast("int"))
    min_boxoffice = df.agg(min('boxoffice')).collect()[0][0]
    df = df.withColumn("boxoffice", when(col("boxoffice").isNull(), min_boxoffice).otherwise(col("boxoffice")))

    """
    Plot preprocessing.
    """

    df = df.withColumn("plot", lower(col("plot")))

    """
    Preprocess the imdb votes.
    """

    df = df.withColumn('imdb_votes', regexp_replace(col('imdb_votes'), "[,]", "").cast('int'))

    """
    Awards transformed into True or False
    """

    df = df.withColumn('awards', my_udf(df['awards']))

    """
    Substitute numVotes value when null with the value of imdb votes
    """

    #df = df.withColumn('numVotes', when(col('numVotes').isNull(), col('imdb_votes')).
                       #otherwise(col('numVotes')))
    df = df.withColumn('startYear', when(col('startYear').isNull(), col('imdb_year')).
                       otherwise(col('startYear')))

    """
    Transformations from string to lists, removing the spaced and then putting everything to lower and then
    splitting.
    """

    df = df.withColumn('country', split(lower(regexp_replace(col('country'), ', ', ',')), ','))
    df = df.withColumn('genre', split(lower(regexp_replace(col('genre'), ', ', ',')), ','))
    df = df.withColumn('actors', split(lower(regexp_replace(col('actors'), ', ', ',')), ','))

    """
    Doing one hot encoding for the genre column
    """

    unique_values = [str(row[0]) for row in df.selectExpr("explode(array_distinct(genre))").distinct().collect()]
    for value in unique_values:
        df = df.withColumn(value, array_contains('genre', value).cast('int'))

    """
    Dropping the columns.
    """

    df = df.drop('_c0', 'primaryTitle', 'originalTitle', 'endYear', 'language', 'rating', 'entry_type', 'production', 'imdb_year', 'genre', 'actors')

    return df

In [148]:
"""
Preprocessing the training dataset.
"""

training_prep = preprocessing_method(training)

In [149]:
"""
Preprocessing the validation dataset.
"""

validation_prep = preprocessing_method(validation)

In [150]:
"""
Preprocessing the test dataset.
"""

test_prep = preprocessing_method(test)

In [151]:
"""
Saving to csv the training dataset.
"""

training_prep.toPandas().to_csv('../datasets/models_dataset/train.csv', index=False)

In [152]:
"""
Saving to csv the validation dataset.
"""

validation_prep.toPandas().to_csv('../datasets/models_dataset/validation.csv', index=False)


In [153]:
"""
Saving to csv the test dataset.
"""

test_prep.toPandas().to_csv('../datasets/models_dataset/test.csv', index=False)